In [ ]:
import json
import re
import requests

from PIL import Image
from time import sleep
from tqdm import tqdm

In [ ]:
page_num = 1
result_urls = []

while True:
    print(f'page #{page_num}')
    res = requests.get(
        f'https://app.astrobin.com/api/v2/iotd/iotd-archive/?page={page_num}',
        headers={'Content-Type': 'application/json'}
    )
    iotd_source = re.findall(r'\<\!DOCTYPE html\>([\w\W\d\D\s\S]+)\<\/span\>(\{[\w\W\d\D\s\S]+\})</pre>', res.text)
    iotd_object = json.loads(iotd_source[0][1].replace('"', '\\"').replace('&quot;', '"'))
    page_urls = [re.findall(r'href=\"(.+)\"', r['image']['thumbnails'][-1]['url'])[0] for r in iotd_object['results']]
    result_urls.extend(page_urls)
    if iotd_object['next'] is None:
        break
    page_num += 1

In [ ]:
result_urls = [re.findall(r'(.+?)\"', url)[0] for url in result_urls]

In [ ]:
for url in tqdm(result_urls):
    image = Image.open(requests.get(url, stream=True).raw).convert('RGB')
    image.save('astrobin_iotd/' + url.replace('https://cdn.astrobin.com/thumbs/', ''))
    sleep(0.25)